## CSC-609/507: Freightful Friday, 4 Nov., 2021


- <em>author:</em> 
 - Burton Rosenberg
 - University of Miami
- <em>last-update:</em>
    
 - 4 nov 2021 -bjr: created
 - 7 nov 2021 -bjr: grammar and details




These are notes about the discrete log problem, and the extraction of information
about the discret log using the quadratic character.

A number of cryptosystems base their security on the difficulty of a problem in
discrete logs over some finite alebraic structure. For instance, the integers modulo
an odd prime p, denoted __Zp__.

In __Zp__ there exist elements called generators, because the collection of all powers 
of g, taken modulo p, yield all p-1 elements 1 through p-1.

$$
\{\, g^i,|\, z\in \mathcal{Z} \,\} = \mathcal{Z}_p^*
$$

The star notation indicates we are considering only the intertible elements 
of __Zp__, that is, only the non-zero elements of __Zp__. 

A generator is this set of numbers including the order in which they are generated.
I denote by $\langle\,g\,\rangle$ the powers of g in order of generation. 

This is an exponential map,

$$
\begin{array}[cccc]
\mbox{exp}:&\mathcal{Z}_{p-1} &\rightarrow& \mathcal{Z}_p^*\\
&i &\mapsto& g^i \pmod{p}
\end{array}
$$

The discrete log to the base g is the inverse of this map,

$$
\begin{array}[cccc]
\mbox{dlog}:&\mathcal{Z}_p^* &\rightarrow& \mathcal{Z}_{p-1}\\
&g^i\pmod{p} &\mapsto& i 
\end{array}
$$

#### Inverses

The extended GCD algorithm efficiently calculates the solution to the equation,

$$
s\,a + t\,b = \mbox{gcd}\,(a,b)
$$

where a and b are given, and s and t are a solution, and all numbers are integers.

In the case that p is a prime, then for all a which would be non-zero in __Zp__, the
solution reduces to, 

$$
s\,a = 1 \pmod{p}
$$

which both shows a is invertible and gives a way to quickly calculate the inverse.

#### Exponentiation and Logarithms

The exponential map, given above, can be efficiently implemeneted, for instance, by 
repeated squaring. The possibility of efficiently implementing the logarithm map, 
however, is the topic of discussion. 

It can be brute-forced, by walking through the powers of g. This is an exponential
time algorithm, because the size of the problem is the length of the descripton of
p, not the value p. That is, a prime  less than 1024 can be described in 10 steps,
reciting the value of each of 10 bits; however the brute-force search for a
logarithm could loop over 1023 attempts, each loop contributing at least one step.

#### Quadratic Character.

Fermat's theorem for primes p, states,

$$
\forall a \in \mathcal{Z}_p, \; a^{p-1}=1 \pmod{p}
$$

If p is odd, and knowing that there are exactly two square roots to 1, modulo p,

$$
\begin{array}[rccc]
 \mbox{ \psi}:  & \mathcal{Z}_p^* & \rightarrow & \{\,1,-1\,\}\\
 &x^{(p-1)/2}\pmod{p} &\mapsto& i 
\end{array}
$$

If the discrete log of a number $a$ even, $a = g^{2i}$ then $g^i$ is a square root of $a$, hence
$a$ has a square root. Legendre's theorem leads quckly to the conclusion that if the discrete
log is odd, the number must not have a square root. It is also noted, that whether a number is or is
not a square root does not depend on which generator is chosen for the logarithm (the base of the
logarithm).

Numbers with square
roots are called _quadratic residues_, and those without, _non-quadratic residues_.

This can break confidentiality in several ways. 

1. For Diffie-Hellman key exchange, it is easy to tell whether the last bit of Alice's or Bob's secret is one or zero. 
1. For the Decisional Diffie Hellman assumption; because the easily computed predicate 
$\bigl(QR(x)\lor QR(y) == QR(z)\bigr)$
is true with probability 1/2 for random $x,y, z$ but always always true when  $x, y, z$ are
a DH triple.


#### Extended attack (the easy bits)

The least significant bit of the discrete log is equal to the easily computed quadratic character
of the number. Writing out the order of __Zp__,

$$
q\,2^s = (p-1)
$$

with $q$ odd, the least significant $s$ bits are easy. Given descrete log $d$, written out in binary, we
can attempt to shift down the bits to read them out one by one as the least significant bit.

To do this, we need to zero the least significant of $d$ and then divide it by 2; this corresponds to 
a possible division by $g$ and a square root.

However, there are two square roots and they differ by a factor of -1. The discrete log of -1 
is $(p-1)/2 = q\,2^{s-1}$. So the two square roots agree on the bottom $s-1$ bits, and hence the first
$s$ bits iteratively extracted by a repeated adjust-then-square root loop, will 
be the lowest $s$ bits of $d$.

_That is a little tricky:_ The $s$-th bit is changed by multiplication of -1, but by then we have already shifted down one binary place.

However all the higher bits are either flipped or not, depending on which of the two square roots
was extracted. If we cannot efficiently determine this, then these higher bits are effectively
ranomized.

#### Extensions to other primes dividing p-1.

The attack can be extended to any prime q that divided $p-1$. Writing again, 

$$
r\,q^s = (p-1)
$$

then consider the discrete log written in base $q$, 

$$
d = \sum_{i\ge 0} b_i q^i
$$

and consider what happens when raised to the power $(p-1)/q$,

\begin{eqnarray*}
(g^d)^{(p-1)/q} &=& (g^{\sum_{i\ge 0} b_i q^i})^{(p-1)/q} \\
&=& g^{b_0(p-1)/q}(g^{\sum_{i\ge 1}b_uq^{i-1}})^{(p-1)} \\
&=& (g^{(p-1)/q})^{b_0} 
\end{eqnarray*}

which takes on only one of three possible values, according to $b_0\in\{\,0,1,\ldots,q-1\,\}$. 
Identify which by trial and error, and then divide off to have,

$$
d' = \sum_{i\ge 1} b_i q^i
$$

and consider what happens when $g^{d'}$ is raise to the power $(p-1)/q^2$. Note however
we need $q^2\,|\,(p-1)$. But if so, then all the arguments work so that we will have,

$$
(g^{d'})^{(p-1)/q^2} = (g^{(p-1)/q})^{b_1} 
$$

and $b_1$ can be identified by trial and error.

As long as $q$ is small enough that trial and error is practical, the $s$ lower order base $q$
digits are all vulnerable to disclosure. This gives rise to the practical advice that the discrete
log find a large prime subgroup of __Zp__, 

In [6]:

class ModP:

    @staticmethod
    def exp_mod(a,b,p):

        a = a%p
        b = b%(p-1)

        if a==0:
            return 0
        if a==1:
            return 1
        if b==0:
            return 1
        if b==1:
            return a
        assert b>1

        if b%2==1:
            return a * ModP.exp_mod(a,b-1,p)%p
        t = ModP.exp_mod(a,b//2,p)
        return (t*t)%p

    @staticmethod
    def qr_p(a,p):
        return ModP.exp_mod(a,(p-1)//2,p)==1

    @staticmethod
    def extended_gcd(a,b):
        """
        extended GCD algorithm. recursive.
        returns (d,s,t) where d = s*a+t*b 
        and d = gcd(a,b)
        """
        assert(
            a>=0 and b>=0 )
        if b==0:
            return (a,1,0)
        (q,r) = divmod(a,b)
        (d,s,t) = ModP.extended_gcd(b,r)
        # gcd(a, b) == gcd(b, r) == s*b + t*r == s*b + t*(a - q*b)
        return (d,t,s-q*t)

    @staticmethod
    def invert(a,p):
        (d,t,s) = ModP.extended_gcd(a,p)
        assert 1==d
        return t%p

    @staticmethod
    def gen_p(g,p,verbose=False):
        r = g 
        for i in range(1,p-1):
            if verbose: print (f'{i}:\t{r}')
            if r==1:
                return False
            r = (r*g)%p
        if verbose: print (f'{p-1}:\t{r}')
        return True

    @staticmethod
    def find_gen(p):
        for g in range(2,p-1):
            if ModP.gen_p(g,p):
                return g
        assert False

    @staticmethod
    def brute_log(x,g,p):
        x = x%p
        assert x!=0

        r = g
        if x==1:
            return 0
        for i in range(1,p-1):
            if r==x:
                return i
            r = r*g%p
        assert False

  
    

In [7]:
class TonelliShanks:
    
    def __init__(self,p):
        self.p = p

    def find_qnr(self):
        for i in range(2,self.p-1):
            if not ModP.qr_p(i,self.p):
                return i
        assert False

    def make_one(self,t):
        i = 1
        e = (t*t)%self.p
        while e!=1:
            i += 1
            e = (e*e)%self.p
        return i

    @staticmethod
    def s_q_form(n):
        i = 0
        while n%2==0:
            i += 1
            n //= 2
        return (i,n)

    def sq_root(self,n):
        if not ModP.qr_p(n,self.p):
            return 0 

        (s,q) = TonelliShanks.s_q_form(self.p-1)
        z = self.find_qnr()
        m = s
        c = ModP.exp_mod(z,q,self.p)
        t = ModP.exp_mod(n,q,self.p)
        r = ModP.exp_mod(n,((q+1)//2),self.p)
        while True:
            if t==0: return 0
            if t==1: return r
            i = self.make_one(t)
            b = ModP.exp_mod(c,2**(m-i-1),self.p)
            m = i
            c = ModP.exp_mod(b,2,self.p)
            t = (t*c)%self.p
            r = (r*b)%self.p
        assert False

class EasyBits:
    
    def __init__(self,p,g):
        self.p = p
        self.g = g
        self.inv_g = ModP.invert(g,p)
        (s,t) = TonelliShanks.s_q_form(p-1)
        self.ts = TonelliShanks(p)
        self.s = s
        self.t = t

    def how_many(self):
        print(f'there are {self.s} easy bits')
        
    def easy_bits(self,n):
        eb = []
        for i in range(self.s):
            if ModP.qr_p(n,self.p):
                eb = [0]+eb
            else:
                eb = [1]+eb
                n = n*self.inv_g % self.p
            n = self.ts.sq_root(n)
        return eb
            
      

In [8]:
my_prime = 11
my_gen = ModP.find_gen(my_prime)
print(f'<{my_gen}> = Z/{my_prime}Z')
ModP.gen_p(my_gen,my_prime,verbose=True)

tonelli_shanks = TonelliShanks(my_prime)
for i in range(1,my_prime):
    sq = tonelli_shanks.sq_root(i)
    if sq!=0:
        assert (sq*sq)%my_prime==i
        print(f'sqrt({i})=\t{sq}, {(my_prime-sq%my_prime)}')

<2> = Z/11Z
1:	2
2:	4
3:	8
4:	5
5:	10
6:	9
7:	7
8:	3
9:	6
10:	1
sqrt(1)=	1, 10
sqrt(3)=	5, 6
sqrt(4)=	9, 2
sqrt(5)=	4, 7
sqrt(9)=	3, 8


In [9]:
      
my_prime = 13
my_gen = ModP.find_gen(my_prime)        
eb = EasyBits(my_prime,my_gen)

ModP.gen_p(my_gen,my_prime,verbose=True)

for n in range(1,my_prime):
    print(n,eb.easy_bits(n),bin(ModP.brute_log(n,my_gen,my_prime)))

1:	2
2:	4
3:	8
4:	3
5:	6
6:	12
7:	11
8:	9
9:	5
10:	10
11:	7
12:	1
1 [0, 0] 0b0
2 [0, 1] 0b1
3 [0, 0] 0b100
4 [1, 0] 0b10
5 [0, 1] 0b1001
6 [0, 1] 0b101
7 [1, 1] 0b1011
8 [1, 1] 0b11
9 [0, 0] 0b1000
10 [1, 0] 0b1010
11 [1, 1] 0b111
12 [1, 0] 0b110
